In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [43]:
train=pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')

In [44]:
test=pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

In [45]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB


In [47]:
train['Fertilizer Name'].value_counts()


Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

In [48]:
train['Soil Type'].value_counts()


Soil Type
Sandy     156710
Black     150956
Clayey    148382
Red       148102
Loamy     145850
Name: count, dtype: int64

In [49]:
train['Crop Type'].value_counts()


Crop Type
Paddy          85754
Pulses         78180
Cotton         69171
Tobacco        68000
Wheat          66628
Millets        65291
Barley         65045
Sugarcane      64269
Oil seeds      64184
Maize          62569
Ground Nuts    60909
Name: count, dtype: int64

In [50]:
from sklearn.preprocessing import LabelEncoder

In [51]:
le=LabelEncoder()

In [52]:
columns_to_encode = ['Soil Type','Crop Type','Fertilizer Name']

In [53]:
for column in columns_to_encode:
    train[column] = le.fit_transform(train[column])

In [54]:
columns_to_encodes = ['Soil Type','Crop Type']

In [55]:
for column in columns_to_encodes:
    test[column] = le.fit_transform(test[column])

In [56]:
X=train.drop(labels=['id','Fertilizer Name'],axis=1)

In [57]:
Y=train['Fertilizer Name']

In [58]:
X.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,37,70,36,1,8,36,4,5
1,27,69,65,4,4,30,6,18
2,29,63,32,4,4,24,12,16
3,35,62,54,4,0,39,12,4
4,35,58,43,3,6,37,2,16


In [59]:
Y.head()

0    4
1    4
2    2
3    0
4    5
Name: Fertilizer Name, dtype: int64

In [60]:
print(X.shape)
print(Y.shape)

(750000, 8)
(750000,)


In [61]:
from imblearn.under_sampling import RandomUnderSampler

In [62]:
rus = RandomUnderSampler(random_state=42)

In [63]:
X_resampled, y_resampled = rus.fit_resample(X, Y)

In [64]:

from sklearn.model_selection import train_test_split

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [66]:
import xgboost as xgb

In [67]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [68]:
num_classes = len(set(Y))
params = {
    'objective': 'multi:softmax',  # or 'multi:softprob' for probabilities
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'max_depth': 3,
    'eta': 0.1,
    'seed': 42
}

In [69]:
bst = xgb.train(params, dtrain, num_boost_round=100)

In [70]:
preds = bst.predict(dtest)

In [71]:

from sklearn.metrics import classification_report, accuracy_score

In [72]:
print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))

Accuracy: 0.17397325988053605

Classification Report:
               precision    recall  f1-score   support

           0       0.17      0.13      0.15     18413
           1       0.17      0.13      0.15     18449
           2       0.18      0.15      0.16     18481
           3       0.17      0.15      0.16     18326
           4       0.17      0.15      0.16     18540
           5       0.18      0.24      0.21     18550
           6       0.17      0.26      0.21     18485

    accuracy                           0.17    129244
   macro avg       0.17      0.17      0.17    129244
weighted avg       0.17      0.17      0.17    129244



In [79]:
print(X_test.shape[0])   # Should be 250000
print(preds.shape[0])    # Should also be 250000


250000
250000


In [80]:
ids = test["id"]

# Drop 'id' to get feature matrix
X_test = test.drop("id", axis=1)

# Convert to DMatrix
dtest = xgb.DMatrix(X_test)

# Predict using the trained model
preds = bst.predict(dtest)  # Use .argmax(axis=1) if using softprob

preds = preds.astype(int)

class_names = ['14-35-14', '10-26-26', '17-17-17','28-28','20-20','DAP','Urea']  # Replace with your actual class names
le = LabelEncoder()
le.fit(class_names)
pred_labels = le.inverse_transform(preds)

# Create submission DataFrame
submission = pd.DataFrame({
    "id": ids,
    "Fertilizer Name": pred_labels  # Ensure it's integer labels
})

# Save to CSV
submission.to_csv("submissiof.csv", index=False)

print("Submission file 'submission.csv' created successfully!")

Submission file 'submission.csv' created successfully!
